# Modelo de Clasificación PQR nivel 1 con Word2Vec

Este libro aborda el preprocesamiento, y el entrenamiento de un modelo de clasificación para PQRs

### Exploración y Preprocesamiento

En esta primera sección se explorará el set de datos y se preprocesarán los textos con los cuales se entrenará el modelo

In [1]:
## importamos las librerias
##
import pandas as pd
import numpy as np
import re
import gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('PQRs_2018-nov2020.csv', sep=';', encoding='utf-8')

C:\Users\juand\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,10,11,14,15,18,19,20,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head(3)

,ID,forma_requerimiento,identificacion,solicitante,dir_solicitante,tipo_solicitud,cuenta,fec_solicitud,fec_vencmto,fec_rpta,...,TELEFONO,CELULAR,EMAIL,CIRCUITO,NODO,UBICACION,CLASE_SERVICIO,ANIOMES,Tipo fuente,DOCUMENTO
0,PQR_201901_1,verbal,25161887,OFIR DE MARIA CORREA CORREA,CRA 16 18 06,peticion,101202096,8/01/2019,28/01/2019,8/01/2019,...,NaN,3.00981e+09,NaN,HER23L12,D20299,U,Residencial,201901,PQR,NaN
1,PQR_201901_2,verbal,25155655,AURA ALICIA MEJIA CUERVO,CLL 19 15 74,peticion,101249151,4/01/2019,25/01/2019,4/01/2019,...,3.64131e+06,NaN,NaN,HER23L15,D23538,U,Residencial,201901,PQR,NaN
2,PQR_201901_3,verbal,25161763,AMPARO QUICENO GARCIA,CLL 20 15 15 PSO 2,peticion,101274963,16/01/2019,5/02/2019,16/01/2019,...,NaN,3.11739e+09,NaN,HER23L15,D23553,U,Residencial,201901,PQR,NaN


In [4]:
df['forma_requerimiento'].value_counts()

verbal                       575254
telefonico                    57263
escrito                        8530
pagina web                     6238
e-mail                         1607
asesor remoto                   762
chat                            408
otros medios electronicos        92
sspd                             17
interventoria                     3
Name: forma_requerimiento, dtype: int64

In [5]:
for i in range(15):
    print(df[df['forma_requerimiento']=='e-mail']['OBSERVACION USUARIO'].iloc[i])
    print('\n')

#df_copy[df_copy['tipo_solicitud']==s]['tipo_solicitud_1']

2019022000003 - REVISIÓN COMPLETA AL MEDIDOR -  EL NÚMERO DE CUENTA 120210036, EN DONDE INEXPLICABLEMENTE SE PRESENTÓ UN INCREMENTO EN LA FACTURA POR UN SUPUESTO INCREMENTO DE CONSUMO, EXPUESTO POR USTEDES DE LA MISMA MANERA QUE EL PRESENTE INCIDENTE, LO CUAL ME LLEVA A PENS


RAD 20190220000530 - ADJUNTO DOCUMENTO PARA MANTENER LA EXENCIÓN A LA CONTRIBUCIÓN, TENIENDO EN CUENTA NUESTRA ACTIVIDAD PRINCIPAL.


RAD - 20180220019579 - REALIZAR VISITA PARA VERIFICAR ESTADO DEL SISTEMA E IDENTIFICAR EL POR QUE DEL INCREMENTO DE LA FACTURACIÓN Y HACER


RAD - 20180220020853-,  ADJUNTO RUT ACTUALIZADO PARA BENEFICIO DE EXONERACIÓN DE CONTRIBUCIÓN POR SER INDUSTRIA


RAD - 20180220020853 - ADJUNTO RUT ACTUALIZADO PARA BENEFICIO DE EXONERACIÓN DE CONTRIBUCIÓN POR SER INDUSTRIA


RAD - 20190220000329 - BUENAS TARDES EL DÍA 10 DEL MES EN CURSO A LA VECINA DESAFORTUNADAMENTE SE LE ENCIENDE LA CASA, COMO EL CABLE DE LA ENERGÍA QUE USTEDES PUSIERON PASA POR EL TECHO ESTE CABLE QUE ALIMENTA MI CASA SE

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650175 entries, 0 to 650174
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     650175 non-null  object 
 1   forma_requerimiento    650174 non-null  object 
 2   identificacion         650153 non-null  object 
 3   solicitante            650098 non-null  object 
 4   dir_solicitante        650145 non-null  object 
 5   tipo_solicitud         650175 non-null  object 
 6   cuenta                 650175 non-null  int64  
 7   fec_solicitud          650175 non-null  object 
 8   fec_vencmto            650175 non-null  object 
 9   fec_rpta               650175 non-null  object 
 10  clase_inicial          343509 non-null  object 
 11  estrato                650175 non-null  object 
 12  a_favor_de             650175 non-null  object 
 13  tipo_solicitud_1       650175 non-null  object 
 14  documento              262317 non-nu

In [7]:
## cantidad total de filas
## 
len(df)

650175

In [8]:
df = pd.concat([df[df['forma_requerimiento']=='pagina web'], df[df['forma_requerimiento']=='e-mail']], ignore_index=True)
df.head(3)

,ID,forma_requerimiento,identificacion,solicitante,dir_solicitante,tipo_solicitud,cuenta,fec_solicitud,fec_vencmto,fec_rpta,...,TELEFONO,CELULAR,EMAIL,CIRCUITO,NODO,UBICACION,CLASE_SERVICIO,ANIOMES,Tipo fuente,DOCUMENTO
0,PQR_201901_1887,pagina web,10082846,DIEGO FRANCO BARCO,BRR MARACAY MNZ B CAS 4,peticion,152991693,28/01/2019,15/02/2019,31/01/2019,...,3.30308e+06,NaN,NaN,CAM23L12,D12071,U,Residencial,201901,PQR,NaN
1,PQR_201901_1912,pagina web,51655066,LUZ MARINA REYES,MNZ 27 CAS 40 CAMPESTRE D,queja,153997234,28/12/2018,21/01/2019,10/01/2019,...,6.88633e+07,3.18304e+09,NaN,CAM23L12,D12032,U,Residencial,201901,PQR,NaN
2,PQR_201901_1936,pagina web,30285814,AMPARO CASTRO OSORIO,DIG 19A 8T 31 CAMPESTRE B,peticion,154952692,4/01/2019,25/01/2019,16/01/2019,...,6.32201e+07,3.14475e+09,NaN,CAM23L12,D12150,U,Residencial,201901,PQR,NaN


In [9]:
df['tipo_solicitud'] = df['tipo_solicitud'].map(lambda x: 'recurso' if x=='recurso reposicion en subsidio de apelacion' or x=='recurso de reposicion' else x)

In [10]:
df_copy = df.copy()

In [11]:
## Eliminacion de NaN
## 
df_copy['OBSERVACION USUARIO'].isna().value_counts()

False    7164
True      681
Name: OBSERVACION USUARIO, dtype: int64

In [12]:
## Eliminacion de elementos sin sentido
##
df_copy['OBSERVACION USUARIO'] = df_copy['OBSERVACION USUARIO'].map(lambda x: str(x))
df_copy['OBSERVACION USUARIO'] = df_copy['OBSERVACION USUARIO'].map(lambda x: x.lower())
len_obs_usuario = [(i, len(o)) for i, o in enumerate(df_copy['OBSERVACION USUARIO'])]

In [13]:
## Eliminacion de elementos sin sentido
## lista con las observaciones que tengan una longitud de observación menor a
## x caracteres.
##
len_menor_x = [(i,o) for i, o in sorted(len_obs_usuario, key=lambda t: t[1]) if o <=20]
index_menor_x = [i for i, o in len_menor_x]

In [14]:
len(index_menor_x)

766

In [15]:
# sorted(len_obs_usuario, key= lambda t: t[1])

In [16]:
## Eliminacion elementos sin sentido
## se procede a eliminarlas
##
df_copy.drop(index_menor_x, inplace=True)

In [17]:
df_copy = df_copy.reset_index(drop=True)
#len_obs_usuario = [(i, len(o)) for i, o in enumerate(df_copy['OBSERVACION USUARIO'])]
#sorted(len_obs_usuario, key= lambda t: t[1])

In [18]:
df_copy['len_obs'] = df_copy['OBSERVACION USUARIO'].map(lambda x: len(x))

In [19]:
df_copy['len_obs'].describe()

count    7079.000000
mean      209.921034
std       155.858657
min        21.000000
25%       100.000000
50%       180.000000
75%       286.000000
max      1000.000000
Name: len_obs, dtype: float64

In [20]:
#sns.set_theme()

#sns.relplot(data=df_copy['len_obs'], kind='line')

#df_copy.plot(y='len_obs', kind='line')
#plt.show()

In [21]:
count = 0
for l in df_copy['len_obs']:
    if l >=500:
        count += 1
print(f"{count} es el numero de observaciones se sobrepasan los 500 caracteres, equivalente a un "
     f"{round(count/len(df_copy), 4)}%")

309 es el numero de observaciones se sobrepasan los 500 caracteres, equivalente a un 0.0437%


In [22]:
## Eliminacion elementos sin sentido
## se guardan las observaciones para ver de que se tratan
##
# len_menor_x_obs = [df_copy['OBSERVACION USUARIO'][i] for i, o in len_menor_x]

In [23]:
## Veamos los tipos de solicitudes nivel 1
##
df_copy['tipo_solicitud'].value_counts()

peticion                           3991
reclamo                            1941
queja                               587
pedidos/proyectos                   278
recurso                             277
programa de financiacion social       5
Name: tipo_solicitud, dtype: int64

In [24]:
## Vamos a depurar estos tipos de solicitudes.
##
df_copy['tipo_solicitud'] = df_copy['tipo_solicitud'].map(lambda x: 'peticion' if x=='programa de financiacion social' or x=='autogestion' else x)

In [25]:
## Validamos el cambio
##
df_copy['tipo_solicitud'].value_counts()

peticion             3996
reclamo              1941
queja                 587
pedidos/proyectos     278
recurso               277
Name: tipo_solicitud, dtype: int64

In [26]:
tipos_solicitud = df_copy['tipo_solicitud'].unique().tolist()
for s in tipos_solicitud:
    print(f"{s}: \n")
    print(df_copy[df_copy['tipo_solicitud']==s][['tipo_solicitud_1']].value_counts())
    print("\n")

peticion: 

tipo_solicitud_1                                            
exencion contribucion                                           1591
informacion general                                              764
cambio de datos basicos                                          413
terminacion del contrato                                         161
gestionar financiacion                                           130
perjuicios por falla en la prestacion del servicio               120
trabajos tecnicos en medidores, transformadores, a               111
cambio datos basicos                                             108
requerimiento competencia de otra empresa                         92
devolucion o traslado de saldo a favor                            67
trabajos tecnicos en med, trafos, acometidas o en infraestr.      58
traslado de deuda                                                 57
emision de documento para pago                                    42
separacion temporal o pago dir

In [27]:
sorted(df_copy[df_copy['tipo_solicitud']=='peticion']['tipo_solicitud_1'].unique().tolist())

['actualizacion por habeas data',
 'aplicacion beneficio propiedad o venta de activos',
 'cambio datos basicos',
 'cambio de datos basicos',
 'cambio de plazo en la financiacion pfs',
 'cambio de uso',
 'condicion especial por solicitud del propietario',
 'condicion especial solicitud propietario',
 'dano en equipo electrico',
 'danos en equipo electrico',
 'descongelados covid-19',
 'desenergizacion de infraestructura',
 'devolucion o traslado de saldo a favor',
 'devolucion y/o copia de documentos',
 'direccion incorrecta',
 'emision de documento para pago',
 'estado deuda pfs',
 'exencion contribucion',
 'facturacion a la fecha',
 'gestionar financiacion',
 'informacion general',
 'inscripcion, retiro o bloqueo de tarjeta pfs',
 'modificacion de infraestructura por incumplimiento',
 'modificar cobros de terceros',
 'normalizar instalacion',
 'pago de la factura de otro usuario',
 'pago parcial',
 'perjuicios por falla en la prestacion del servicio',
 'poda de arboles',
 'reclamacion

In [28]:
## hay un monton de tipos de solicitudes nivel 2 que realmente son las mismas
## ej: 'revision, mntto, remodelacion o expansion infraestruc. a.p.' ==  'revision, mtto, remodelacion o expansion a.p'
## vamos a estandarizar esto.
##

In [29]:
import Levenshtein as lev

In [30]:
tipo_sol_nv2_p = []
tipo_sol_nv2_r = []
tipo_sol_nv2_pro = []
tipo_sol_nv2_q = []
tipo_sol_nv2_rso = []
for s in tipos_solicitud:
    temp = sorted(df_copy[df_copy['tipo_solicitud']==s]['tipo_solicitud_1'].unique().tolist())
    for i in range(len(temp) - 1):
        a = temp[i]
        b = temp[i+1]
        ratio = lev.ratio(a, b)
        if ratio >= 0.75:
            if s == 'peticion':
                if a not in tipo_sol_nv2_p:
                    tipo_sol_nv2_p.append(a)
            elif s == 'reclamo':
                if a not in tipo_sol_nv2_r:
                    tipo_sol_nv2_r.append(a)
            elif s == 'pedidos/proyectos':
                if a not in tipo_sol_nv2_pro:
                    tipo_sol_nv2_pro.append(a)
            elif s == 'queja':
                if a not in tipo_sol_nv2_q:
                    tipo_sol_nv2_q.append(a)
            else:
                if a not in tipo_sol_nv2_rso:
                    tipo_sol_nv2_rso.append(a)
        else:
            if s == 'peticion':
                if b not in tipo_sol_nv2_p:
                    tipo_sol_nv2_p.append(b)
            elif s == 'reclamo':
                if b not in tipo_sol_nv2_r:
                    tipo_sol_nv2_r.append(b)
            elif s == 'pedidos/proyectos':
                if b not in tipo_sol_nv2_pro:
                    tipo_sol_nv2_pro.append(b)
            elif s == 'queja':
                if b not in tipo_sol_nv2_q:
                    tipo_sol_nv2_q.append(b)
            else:
                if b not in tipo_sol_nv2_rso:
                    tipo_sol_nv2_rso.append(b)

In [31]:
len(sorted(df_copy[df_copy['tipo_solicitud']=='peticion']['tipo_solicitud_1'].unique().tolist()))

46

In [32]:
len(tipo_sol_nv2_p)

41

In [33]:
tipo_sol_nv2_p.append('actualizacion por habeas data')
tipo_sol_nv2_p = sorted(tipo_sol_nv2_p)
tipo_sol_nv2_p

['actualizacion por habeas data',
 'aplicacion beneficio propiedad o venta de activos',
 'cambio datos basicos',
 'cambio de plazo en la financiacion pfs',
 'cambio de uso',
 'condicion especial por solicitud del propietario',
 'dano en equipo electrico',
 'descongelados covid-19',
 'desenergizacion de infraestructura',
 'devolucion o traslado de saldo a favor',
 'devolucion y/o copia de documentos',
 'direccion incorrecta',
 'emision de documento para pago',
 'estado deuda pfs',
 'exencion contribucion',
 'facturacion a la fecha',
 'gestionar financiacion',
 'informacion general',
 'inscripcion, retiro o bloqueo de tarjeta pfs',
 'modificacion de infraestructura por incumplimiento',
 'modificar cobros de terceros',
 'normalizar instalacion',
 'pago de la factura de otro usuario',
 'pago parcial',
 'perjuicios por falla en la prestacion del servicio',
 'poda de arboles',
 'reclamacion extemporanea o agotada la via administ',
 'reconexion',
 'reinstalacion',
 'requerimiento competencia 

In [34]:
tipo_sol_nv2_p.remove('trabajos tecnicos en med, trafos, acometidas o en infraestr.')

In [35]:
len(sorted(df_copy[df_copy['tipo_solicitud']=='reclamo']['tipo_solicitud_1'].unique().tolist()))

23

In [36]:
len(tipo_sol_nv2_r)

18

In [37]:
tipo_sol_nv2_r.append('clase de uso incorrecto')
tipo_sol_nv2_r = sorted(tipo_sol_nv2_r)
tipo_sol_nv2_r

['clase de uso incorrecto',
 'cobro de acuerdo de pago o financiacion',
 'cobro de cargos relacionados con el servicio publi',
 'cobro de otros bienes o servicios en la factura',
 'cobro de revisiones',
 'cobro desconocido',
 'cobro multiple y/o acumulado',
 'cobro por recuperacion de consumos',
 'cobros inoportunos',
 'cobros por conexion, reconexion, reinstalacion',
 'cobros por promedio',
 'estrato incorrecto',
 'inconformidad con el aforo',
 'inconformidad con el consumo o produccion factura',
 'inconformidad por cobros por normalizacion del ser',
 'pago sin abono a cuenta',
 'solicitud de rompimiento de solidaridad',
 'subsidios y contribuciones',
 'tarifa incorrecta']

In [38]:
len(sorted(df_copy[df_copy['tipo_solicitud']=='pedidos/proyectos']['tipo_solicitud_1'].unique().tolist()))

16

In [39]:
len(tipo_sol_nv2_pro)

14

In [40]:
tipo_sol_nv2_pro.append('asesoria de proyectos')
tipo_sol_nv2_pro.append('revision de proyectos electricos nivel ii y iii')
tipo_sol_nv2_pro = sorted(tipo_sol_nv2_pro)
tipo_sol_nv2_pro

['asesoria de proyectos',
 'cambio de sistema',
 'cancelacion de provisional',
 'liquidacion de conexos proyectos',
 'portafolio instalaciones sin cotizacion',
 'portafolio trabajos a terceros con cotizacion',
 'provisional a definitivo',
 'retiro de servicios temporales',
 'revision de proyectos electricos nivel i',
 'revision de proyectos electricos nivel ii y iii',
 'solicitud cambio de comercializador (exporta)',
 'solicitud de interventoria',
 'solicitud factibilidad del servicio',
 'solicitud participacion expansion',
 'solicitud viabilidad compra/donacion activos',
 'visita puesta en servicio']

In [41]:
len(sorted(df_copy[df_copy['tipo_solicitud']=='queja']['tipo_solicitud_1'].unique().tolist()))

20

In [42]:
len(tipo_sol_nv2_q)

16

In [43]:
tipo_sol_nv2_q.append('afectacion ambiental')
tipo_sol_nv2_q = sorted(tipo_sol_nv2_q)
tipo_sol_nv2_q

['afectacion ambiental',
 'comportamiento funcionario (trabajador-contra)',
 'demora en atender peticion',
 'demora en reparar dano',
 'entrega inoportuna o no entrega de la factura',
 'falla en la prestacion del servicio por calidad',
 'falla en la prestacion del servicio por continuida',
 'inconformidad con el canal de atencion',
 'inconformidad con la informacion',
 'inconformidad en la atencion de condiciones de seg',
 'inconformidad por la programacion de interrupcione',
 'inconformidad punto de venta y/o recaudo',
 'inconformidad trabajos',
 'inconformidad venta portafolio',
 'incumplimiento revision en las fechas establ',
 'levantamiento de desechos vegetales',
 'suspension o corte del servicio']

In [44]:
len(sorted(df_copy[df_copy['tipo_solicitud']=='recurso']['tipo_solicitud_1'].unique().tolist()))

12

In [45]:
tipo_sol_nv2_rso = sorted(df_copy[df_copy['tipo_solicitud']=='recurso']['tipo_solicitud_1'].unique().tolist())
tipo_sol_nv2_rso

['clase de uso incorrecto',
 'cobro de cargos relacionados con el servicio publi',
 'cobro de otros bienes o servicios en la factura',
 'cobros inoportunos',
 'cobros por promedio',
 'inconformidad con el consumo o produccion facturad',
 'negacion del servicio',
 'otros motivos de recurso',
 'pago sin abono a cuenta',
 'solicitud de rompimiento de solidaridad',
 'subsidios y contribuciones',
 'tarifa incorrecta']

In [46]:
## Los siguientes tipos de solicitudes nivel 1 si tuvieron cambion en los
## niveles 2: peticion, reclamo, pedidos/proyectos, queja
## estos se procederan a estadarizarce.
##
def nv2_replacements(solicitud_nv2):
    s = tipo_sol_nv2_p + tipo_sol_nv2_r + tipo_sol_nv2_pro + tipo_sol_nv2_q + tipo_sol_nv2_rso
    s.remove('revision de proyectos electricos nivel i')
    s.remove('revision de proyectos electricos nivel ii y iii')
    s.remove('falla en la prestacion del servicio por calidad')
    s.remove('falla en la prestacion del servicio por continuida')
    for element in s:
        if lev.ratio(element, solicitud_nv2) >= 0.75:
            solicitud_nv2 = element
        else:
            solicitud_nv2
    return solicitud_nv2

In [47]:
df_copy['tipo_solicitud_1'] = df_copy['tipo_solicitud_1'].map(nv2_replacements)

In [48]:
sorted(df_copy['tipo_solicitud_1'].unique().tolist())

['actualizacion por habeas data',
 'afectacion ambiental',
 'aplicacion beneficio propiedad o venta de activos',
 'cambio datos basicos',
 'cambio de comercializador',
 'cambio de plazo en la financiacion pfs',
 'cambio de sistema',
 'cambio de uso',
 'cancelacion de provisional',
 'clase de uso incorrecto',
 'cobro de acuerdo de pago o financiacion',
 'cobro de cargos relacionados con el servicio publi',
 'cobro de otros bienes o servicios en la factura',
 'cobro de revisiones',
 'cobro desconocido',
 'cobro multiple y/o acumulado',
 'cobro por recuperacion de consumos',
 'cobros inoportunos',
 'cobros por conexion, reconexion, reinstalacion',
 'cobros por promedio',
 'comportamiento funcionario (trabajador-contra)',
 'condicion especial por solicitud del propietario',
 'dano en equipo electrico',
 'demora en atender peticion',
 'demora en reparar dano',
 'descongelados covid-19',
 'desenergizacion de infraestructura',
 'devolucion o traslado de saldo a favor',
 'devolucion y/o copia 

In [49]:
df_copy['tipo_solicitud_1'] = df_copy['tipo_solicitud_1'].map(lambda x: 'falla en la prestacion del servicio por continuidad' if x=='falla en la prestacion del servicio por continuida' else x)
df_copy['tipo_solicitud_1'] = df_copy['tipo_solicitud_1'].map(lambda x: 'trabajos tecnicos en med, trafos, acometidas o en infraestr.' if x=='trabajos tecnicos en medidores, transformadores, a' else x)

In [50]:
sorted(df_copy['tipo_solicitud_1'].unique().tolist())

['actualizacion por habeas data',
 'afectacion ambiental',
 'aplicacion beneficio propiedad o venta de activos',
 'cambio datos basicos',
 'cambio de comercializador',
 'cambio de plazo en la financiacion pfs',
 'cambio de sistema',
 'cambio de uso',
 'cancelacion de provisional',
 'clase de uso incorrecto',
 'cobro de acuerdo de pago o financiacion',
 'cobro de cargos relacionados con el servicio publi',
 'cobro de otros bienes o servicios en la factura',
 'cobro de revisiones',
 'cobro desconocido',
 'cobro multiple y/o acumulado',
 'cobro por recuperacion de consumos',
 'cobros inoportunos',
 'cobros por conexion, reconexion, reinstalacion',
 'cobros por promedio',
 'comportamiento funcionario (trabajador-contra)',
 'condicion especial por solicitud del propietario',
 'dano en equipo electrico',
 'demora en atender peticion',
 'demora en reparar dano',
 'descongelados covid-19',
 'desenergizacion de infraestructura',
 'devolucion o traslado de saldo a favor',
 'devolucion y/o copia 

In [51]:
## los resultados de estos cambios
##
print(f"Se pasó de tener {len(df['tipo_solicitud_1'].unique())} solicitudes nivel 2 a {len(df_copy['tipo_solicitud_1'].unique())}")

Se pasó de tener 113 solicitudes nivel 2 a 92


Esta primer parte de exploración dejó como resultado una depuración de registros sin sentido y con poco valor semántico, además de una depuración de clasificaciones nivel 2.

A continuación, se procede con el preprocesamiento

In [52]:
## solo se identifica un bi-grama que no genera valor: ('autogest', 'autogestión')
##
df_copy['OBSERVACION USUARIO'] = df_copy['OBSERVACION USUARIO'].map(lambda x: x.replace('autogest autogestion', ''))

In [53]:
df_copy['OBSERVACION USUARIO'] = df_copy['OBSERVACION USUARIO'].map(lambda x: x.replace('autogest autogestión', ''))

In [54]:
## Se eliminan espacios dobles
##
df_copy['OBSERVACION USUARIO'] = df_copy['OBSERVACION USUARIO'].map(lambda x: x.replace('  ', ' '))

In [55]:
df_copy['tipo_solicitud'].value_counts()

peticion             3996
reclamo              1941
queja                 587
pedidos/proyectos     278
recurso               277
Name: tipo_solicitud, dtype: int64

In [56]:
## ----- A partir de aquí nos centraremos en equilibrar los tipos de solicitudes -----
## la idea acá es balancear un poco la cantidad de registros entre categorias
## las peticiones hay que reducirlas lo máximo posible podemos quedar al menos con
## unas 10,000 al igual que los reclamos tener unos 10,000. Por el lado de los recursos
## se ensayará creando unos registros sintéticos con LSTM.

In [57]:
datos = df_copy[['tipo_solicitud', 'tipo_solicitud_1', 'OBSERVACION USUARIO']]

In [58]:
datos['tipo_solicitud'].value_counts()

peticion             3996
reclamo              1941
queja                 587
pedidos/proyectos     278
recurso               277
Name: tipo_solicitud, dtype: int64

In [59]:
## Exploremos las peticiones y miremos como hacemos para depurarlas

In [60]:
datos_peticiones = datos[datos['tipo_solicitud']=='peticion'][['tipo_solicitud', 'tipo_solicitud_1', 'OBSERVACION USUARIO']]
datos_peticiones.head(3)

,tipo_solicitud,tipo_solicitud_1,OBSERVACION USUARIO
0,peticion,requerimiento competencia de otra empresa,"rad - 20190220001010 - señores chec , en el ba..."
2,peticion,informacion general,20190220000106 - hechos yo amparo castro osori...
3,peticion,emision de documento para pago,rad -+ 20190220000876 - duplicado de factura


In [61]:
datos_peticiones['tipo_solicitud_1'].value_counts()

## para las siguientes categorías se dejan unicamente 1,000 registros por
## categoría:
## emision de documento para pago                                      
## pago parcial                                                        
## separacion temporal o pago dirigido                                  
## informacion general                                                  
## trabajos tecnicos en med, trafos, acometidas o en infraestr.         
## datos basicos                                                        
## reconexion                                                           
## gestionar financiacion                                               
## descongelados covid-19                                               
## exencion contribucion                                                 
## notificacion decision y/o descargos     

exencion contribucion                                           1591
informacion general                                              764
cambio datos basicos                                             521
trabajos tecnicos en med, trafos, acometidas o en infraestr.     169
terminacion del contrato                                         161
gestionar financiacion                                           130
perjuicios por falla en la prestacion del servicio               120
requerimiento competencia de otra empresa                         92
devolucion o traslado de saldo a favor                            67
traslado de deuda                                                 57
emision de documento para pago                                    42
separacion temporal o pago dirigido                               34
requerimientos sspd o sic                                         29
cambio de uso                                                     27
poda de arboles                   

In [62]:
'''peticiones_depurar = ['emision de documento para pago',
                      'pago parcial',
                      'separacion temporal o pago dirigido',
                      'informacion general',
                      'trabajos tecnicos en med, trafos, acometidas o en infraestr.',
                      'datos basicos',
                      'reconexion',
                      'gestionar financiacion',
                      'descongelados covid-19',
                      'exencion contribucion',
                      'notificacion decision y/o descargos',
                      'inscripcion, retiro o bloqueo de tarjeta pfs',
                      'poda de arboles',
                      'cambio de plazo en la financiacion pfs',
                      'terminacion del contrato',
                      'pago de la factura de otro usuario',
                      'condicion especial por solicitud del propietario',
                      'direccion incorrecta',
                      'tarjeta somos (pfs)',
                      'requerimiento competencia de otra empresa',
                      'perjuicios por falla en la prestacion del servicio',
                      'dano en equipo electrico',
                      'devolucion o traslado de saldo a favor']'''

"peticiones_depurar = ['emision de documento para pago',\n                      'pago parcial',\n                      'separacion temporal o pago dirigido',\n                      'informacion general',\n                      'trabajos tecnicos en med, trafos, acometidas o en infraestr.',\n                      'datos basicos',\n                      'reconexion',\n                      'gestionar financiacion',\n                      'descongelados covid-19',\n                      'exencion contribucion',\n                      'notificacion decision y/o descargos',\n                      'inscripcion, retiro o bloqueo de tarjeta pfs',\n                      'poda de arboles',\n                      'cambio de plazo en la financiacion pfs',\n                      'terminacion del contrato',\n                      'pago de la factura de otro usuario',\n                      'condicion especial por solicitud del propietario',\n                      'direccion incorrecta',\n           

In [63]:
peticiones_depurar = ['exencion contribucion',
                     'informacion general',
                     'cambio datos basicos']

In [64]:
import random

for p in peticiones_depurar:
    temp = datos_peticiones[datos_peticiones['tipo_solicitud_1']==p]
    indexes = temp.index
    index_sample = random.sample(list(indexes), len(temp)-180)
    datos_peticiones.drop(index_sample, inplace=True)

In [65]:
datos_peticiones['tipo_solicitud_1'].value_counts()

informacion general                                             180
cambio datos basicos                                            180
exencion contribucion                                           180
trabajos tecnicos en med, trafos, acometidas o en infraestr.    169
terminacion del contrato                                        161
gestionar financiacion                                          130
perjuicios por falla en la prestacion del servicio              120
requerimiento competencia de otra empresa                        92
devolucion o traslado de saldo a favor                           67
traslado de deuda                                                57
emision de documento para pago                                   42
separacion temporal o pago dirigido                              34
requerimientos sspd o sic                                        29
cambio de uso                                                    27
poda de arboles                                 

In [66]:
len(datos_peticiones)

1660

In [67]:
## Exploremos los reclamos y miremos como hacemos para depurarlas

In [68]:
datos_reclamo = datos[datos['tipo_solicitud']=='reclamo'][['tipo_solicitud', 'tipo_solicitud_1', 'OBSERVACION USUARIO']]
datos_reclamo.head(3)

,tipo_solicitud,tipo_solicitud_1,OBSERVACION USUARIO
5,reclamo,inconformidad con el consumo o produccion fact...,20180220021018 - en esta factura se está cobrn...
6,reclamo,"cobros por conexion, reconexion, reinstalacion",20180220020704 - es legal en colombia el cobro...
23,reclamo,inconformidad con el consumo o produccion fact...,20190220000145 - regularmente se paga la factu...


In [69]:
datos_reclamo['tipo_solicitud_1'].value_counts()

inconformidad con el consumo o produccion facturad    1630
cobro de cargos relacionados con el servicio publi      93
pago sin abono a cuenta                                 75
cobros por promedio                                     25
subsidios y contribuciones                              22
cobros por conexion, reconexion, reinstalacion          19
cobro de otros bienes o servicios en la factura         15
estrato incorrecto                                      13
cobro multiple y/o acumulado                            10
tarifa incorrecta                                        7
cobro de acuerdo de pago o financiacion                  6
solicitud de rompimiento de solidaridad                  5
clase de uso incorrecto                                  5
cobro desconocido                                        4
inconformidad por cobros por normalizacion del ser       4
cobro por recuperacion de consumos                       3
inconformidad con la informacion                        

In [70]:
temp = datos_reclamo[datos_reclamo['tipo_solicitud_1']=='inconformidad con el consumo o produccion facturad']
indexes = temp.index
index_sample = random.sample(list(indexes), len(temp)-1300)
datos_reclamo.drop(index_sample, inplace=True)

In [71]:
datos_reclamo['tipo_solicitud_1'].value_counts()

inconformidad con el consumo o produccion facturad    1300
cobro de cargos relacionados con el servicio publi      93
pago sin abono a cuenta                                 75
cobros por promedio                                     25
subsidios y contribuciones                              22
cobros por conexion, reconexion, reinstalacion          19
cobro de otros bienes o servicios en la factura         15
estrato incorrecto                                      13
cobro multiple y/o acumulado                            10
tarifa incorrecta                                        7
cobro de acuerdo de pago o financiacion                  6
solicitud de rompimiento de solidaridad                  5
clase de uso incorrecto                                  5
cobro desconocido                                        4
inconformidad por cobros por normalizacion del ser       4
cobro por recuperacion de consumos                       3
inconformidad con la informacion                        

In [72]:
len(datos_reclamo)

1611

In [73]:
## Ahora sobre los recuros.
## se ensayará creando unos registros sintéticos con LSTM

In [74]:
datos_recurso = datos[datos['tipo_solicitud']=='recurso'][['tipo_solicitud', 'tipo_solicitud_1', 'OBSERVACION USUARIO']]
datos_recurso.head(3)

,tipo_solicitud,tipo_solicitud_1,OBSERVACION USUARIO
921,recurso,solicitud de rompimiento de solidaridad,"* el recurrente interpone recurso, manifestand..."
996,recurso,inconformidad con el consumo o produccion fact...,el cliente manifista inconformidad con la resp...
1015,recurso,inconformidad con el consumo o produccion fact...,el cliente manifiesta inconformidad con el con...


In [75]:
## vamos a remover la puntuacion y los valores numericos de estas observaciones.
## se dejaran los stopwords.
##
datos_recurso['OBSERVACION USUARIO'] = datos_recurso['OBSERVACION USUARIO'].map(lambda x: str(x))
datos_recurso['OBSERVACION USUARIO'] = datos_recurso['OBSERVACION USUARIO'].map(lambda x: x.lower())
datos_recurso['OBSERVACION USUARIO'] = datos_recurso['OBSERVACION USUARIO'].map(lambda x: ' '.join(re.findall(r'[.,;]|[a-záéíóúñ]{2,}', x)))

In [76]:
datos_recurso['len_obs'] = datos_recurso['OBSERVACION USUARIO'].map(lambda x: len(x))
datos_recurso['len_obs'].describe()

count    277.000000
mean     351.194946
std      268.335729
min       24.000000
25%      154.000000
50%      279.000000
75%      459.000000
max      981.000000
Name: len_obs, dtype: float64

In [77]:
## Se eliminan los recursos con len < 50
##
datos_recurso = datos_recurso.reset_index(drop=True)
len_obs_usuario = [(i, len(o)) for i, o in enumerate(datos_recurso['OBSERVACION USUARIO'])]
len_menor_x = [(i,o) for i, o in sorted(len_obs_usuario, key=lambda t: t[1]) if o <=50]
index_menor_x = [i for i, o in len_menor_x]

In [78]:
datos_recurso.drop(index_menor_x, inplace=True)

In [79]:
datos_recurso['len_obs'].describe()

count    271.000000
mean     358.084871
std      267.212421
min       53.000000
25%      160.000000
50%      281.000000
75%      474.500000
max      981.000000
Name: len_obs, dtype: float64

In [80]:
## vamos a crear un nuevo df con todos los df depurados. 
##
datos_pedidos = datos[datos['tipo_solicitud']=='pedidos/proyectos'][['tipo_solicitud', 'tipo_solicitud_1', 'OBSERVACION USUARIO']]
datos_quejas = datos[datos['tipo_solicitud']=='queja'][['tipo_solicitud', 'tipo_solicitud_1', 'OBSERVACION USUARIO']]

In [81]:
datos_md = pd.concat([datos_peticiones, datos_reclamo, datos_recurso, datos_pedidos, datos_quejas],
                    names=['tipo_solicitud', 'tipo_solicitud_1', 'OBSERVACION USUARIO'],
                    ignore_index=True)

In [82]:
'''datos_md.to_csv('PQR_modelo.csv',
                sep=',',
                header=datos_md.columns,
                index=False)'''

"datos_md.to_csv('PQR_modelo.csv',\n                sep=',',\n                header=datos_md.columns,\n                index=False)"

In [83]:
## el data frame con el se ensayará el nuevo modelo luce entonces así
##
datos_md['tipo_solicitud'].value_counts()

peticion             1660
reclamo              1611
queja                 587
pedidos/proyectos     278
recurso               271
Name: tipo_solicitud, dtype: int64

### Entrenamiento del modelo

In [84]:
## normalización de los textos.
##
import re
import spacy
import nltk
nltk.download('stopwords')
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.collocations import *
from string import punctuation
nlp = spacy.load('es_core_news_md')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
#ensayo
df_copy['OBSERVACION USUARIO'][1660]

'hola el reclamo es por el cobro exagerado para un predio que no está habitado por concepto de alumbrado público de 8589, si bien se que deben cobrar ese servicio me parece ilógico que uno que si está habitado pague por dicho concepto 2.600, pido una rebaja sustancial a dicho cobro porfavor'

In [86]:
# ensayo
sent_tokenize(' '.join(re.findall(r'[.,;]|[a-záéíóúñ]{2,}', df_copy['OBSERVACION USUARIO'][1660])))

['hola el reclamo es por el cobro exagerado para un predio que no está habitado por concepto de alumbrado público de , si bien se que deben cobrar ese servicio me parece ilógico que uno que si está habitado pague por dicho concepto .',
 ', pido una rebaja sustancial dicho cobro porfavor']

In [87]:
#def gensim_preprocess_data(text):
#    stop_words = list(stopwords.words('spanish'))  
#    no_value_words = ['fecha', 'valor']
#    for w in no_value_words:
#        stop_words.append(w)
#    text = str(text)
#    text = text.lower()
#    text = ' '.join(re.findall(r'[.,;]|[a-záéíóúñ]{2,}', text))
    
#    sentences = sent_tokenize(text)
#    tokenized_sentences = list([word_tokenize(sentence) for sentence in sentences])
#    for i in range(0, len(tokenized_sentences)):
#         tokenized_sentences[i] = [word for word in tokenized_sentences[i] if word not in punctuation]
#    return tokenized_sentences

In [88]:
# ensayo
#rrr = gensim_preprocess_data(datos_md['OBSERVACION USUARIO'][369])
#rrr

In [89]:
## extraccion de tokens
#tokens = datos_md['OBSERVACION USUARIO'].map(gensim_preprocess_data)
#sentencias = []
#for t in tokens:
#    for s in t:
#        sentencias.append(s)
#sentencias

In [90]:
# el argumento size se refiere a el tamaño de columnas de la matriz resultante
#from gensim.models import Word2Vec
#model = Word2Vec(sentencias, size=300, window=5, min_count=1, workers=4)

In [91]:
## se define el vocabulario para entrenar
#model.build_vocab(sentencias)

In [92]:
## se entrena el modelo word2vec con las sentencias como insumo, y el número total 
## de ejemplos son las mismas sentencias. Se puede definir cualquier número de epochs
##
#model.train(sentences=sentencias, total_examples=len(sentencias), epochs=4000)

In [93]:
#(model.wv.vectors).shape

In [94]:
## se guarda el modelo. (este en particular demoró +/- 40m en el entrenamiento)
##
#model.save("word2vec.model")

In [95]:
## acá vemos el tamaño de la matriz resultante
##
#max_dataset_size = len(model.wv.vectors)

In [96]:
def clean_text(text):
    stop_words = list(stopwords.words('spanish'))  
    no_value_words = ['fecha', 'valor']
    for w in no_value_words:
        stop_words.append(w)
    text = str(text)
    text = text.lower()
    text = ' '.join(re.findall(r'[.,;]|[a-záéíóúñ]{2,}', text))
    
    text = word_tokenize(text)
    for w in stop_words:
        while w in text: text.remove(w)
    return ' '.join(list(dict.fromkeys(text)))

In [97]:
clean_text(df_copy['OBSERVACION USUARIO'][1660])

'hola reclamo cobro exagerado predio habitado concepto alumbrado público , si bien deben cobrar servicio parece ilógico pague dicho . pido rebaja sustancial porfavor'

In [98]:
datos_md['OBSERVACION USUARIO'] = datos_md['OBSERVACION USUARIO'].apply(clean_text)

In [99]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(datos_md, test_size=0.2, random_state=42, stratify=datos_md['tipo_solicitud'])

In [100]:
from gensim.models.doc2vec import TaggedDocument
from nltk.corpus import stopwords

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['OBSERVACION USUARIO']), tags=[r.tipo_solicitud]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['OBSERVACION USUARIO']), tags=[r.tipo_solicitud]), axis=1)

In [101]:
train_tagged.values[30]

TaggedDocument(words=['usuaria', 'solicita', 'explique', 'aumento', 'energía', 'factura', 'anterior', 'actual', 'manifiesta', 'exagerado', 'venido', 'aumentando', 'loca'], tags=['reclamo'])

In [102]:
whole_tagged = [*train_tagged.values, *test_tagged.values]

In [103]:
import multiprocessing

cores = multiprocessing.cpu_count()

In [198]:
from tqdm import tqdm
from gensim.models import Doc2Vec
from gensim.models import Word2Vec

#sents = [registro[0] for registro in train_tagged.values]
#sents = [registro[0] for registro in whole_tagged]
#model_w2v = Word2Vec(sents, size=300, window=5, min_count=0, workers=cores, sg=1)
#model_w2v.build_vocab(sents)

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=1, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████████████████████████████████████████████████████████████████████| 3525/3525 [00:00<00:00, 874330.08it/s]


In [199]:
model_dbow.wv.vectors.shape

(8368, 300)

In [105]:
## Esto basicamente es la cantidad de palabras con las que cuenta el corpus
##

print(f"la cantidad total de palabras es {model_w2v.corpus_total_words}")
print(f"la cantidad total de sentencias es {len(sents)}")
#model_dbow.corpus_total_words

la cantidad total de palabras es 81022
la cantidad total de sentencias es 4407


In [146]:
## Entrenamiento de los modelos
##

from sklearn import utils

'''model_w2v.train(sents,
                total_examples=model_w2v.corpus_count,
                epochs=300)'''

'''for epoch in range(300):
    model_dbow.train(utils.shuffle([x for x in train_tagged.values]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha'''

In [200]:
# for Doc2Vec
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

# for Word2Vec
'''def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc[1], model.wv[doc[0]]) for doc in sents])
    return targets, regressors'''

'def vec_for_learning(model, tagged_docs):\n    sents = tagged_docs.values\n    targets, regressors = zip(*[(doc[1], model.wv[doc[0]]) for doc in sents])\n    return targets, regressors'

In [201]:
from sklearn.svm import LinearSVC

y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

#y_train, X_train = vec_for_learning(model_w2v, train_tagged)
#y_test, X_test = vec_for_learning(model_w2v, test_tagged)

svml = LinearSVC(C=10, random_state=22022021, max_iter=5000)
svml.fit(X_train, y_train)
y_pred = svml.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score

print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.3979591836734694
Testing F1 score: 0.3307842095234457


#### Conclusion

Se aplicaron dos técnicas de extracción de características:
* Distributed Bag of Words (Doc2Vec). La cual arrojó los resultados arriba mostrados.
* Skip-N-Gram (Word2Vec). Con este método de extracción de características no veo como aplicarlo a un entrenamiento de un modelo supervisado, ya que por la representación de las palabras en vectores esto resulta en matrices de diferentes tamaños no procesables por métodos de machine learning. Por ejemplo: una petición que tiene 12 palabras se le pasa w2v para ver la representación numérica de las palabras en el contexto, y esto puede dar una matriz de 12x300 y así por cada PQR, entonces no veo una manera de hacer que esa matriz se represente por un vector de 300x1. A fin de cuentas no creo que sea un método que se use para clasificación supervisada.